In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES = 3

env: CUDA_VISIBLE_DEVICES=3


In [4]:
from batchflow.batchflow.utils import make_initialization_inputs

ImportError: cannot import name 'make_initialization_inputs' from 'batchflow.utils' (/opt/conda/lib/python3.8/site-packages/batchflow/utils.py)

In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import get_module_performance

pd.set_option("display.precision", 2)

ImportError: cannot import name 'make_initialization_inputs' from 'batchflow.utils' (/opt/conda/lib/python3.8/site-packages/batchflow/utils.py)

In [ ]:
device = 'cuda:0'
device

In [ ]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [ ]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device, 
                                                               track_backward=True, memory_unit='KB')

module_collection_stats

In [ ]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

In [ ]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device,
                                                               track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

In [ ]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_performance(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats